In [198]:
import ee
import geemap

In [206]:
#Only have to do this once
ee.Authenticate()

Enter verification code: 4/0QFRh7h8C3a1AnL1RCbV8raixULcIeNCpSRSdU7zjyH-vpQUmFo-Pjk

Successfully saved authorization token.


In [207]:
#Not sure this is necessary every time
ee.Initialize()

In [237]:
#Create Map and center on Manauas EC Flux Tower coordinates
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [238]:
#Create a point for EC Flux Tower
brma2 = ee.Geometry.Point(-60.20929722, -2.609097222)

#Determine the area I want my square buffer to be
areaM2 = 3.087969e9
square = brma2.buffer(ee.Number(areaM2).sqrt().divide(2), 1).bounds()

#Add the polygon to the map
Map.addLayer(square, {}, 'Manauas Buffer')

In [240]:
#Load Sentinel-2 data and filter out cloud cover for tiles with less than 20% coverage
#Note: there is also a cloud masking function that I haven't tried. This was just easy.
sentinel2 = (ee.ImageCollection("COPERNICUS/S2") 
    .filterDate('2015-01-01', '2020-01-01') 
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(square)))

#Create some parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'max': 2000
}

Map.addLayer(sentinel2, vis_params, 'Raw Sentinel')

In [211]:
#Calculate the median to create one image; can only use an image with clip function
s2_median = sentinel2.median()

#Clip median Sentinel-2 data to the square
clipped = s2_median.clip(square)
Map.addLayer(clipped, vis_params, 'Median Sentinel')

In [ ]:
#Calculate NDRE
#Select the least cloudy image
#Note: having to select only one image of the collection to calculate NDRE is REALLY annoying
#I can't find a way around it right now.
image = (ee.Image(
  sentinel2.filterBounds(square)
    .filterDate('2015-01-01', '2020-01-01')
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()))

#Set up NDRE calculation
nir = image.select('B8')
edge = image.select('B5')
ndre = nir.subtract(edge).divide(nir.add(edge)).rename('NDRE')
ndreParams = {min: -1, max: 1, 'palette': ['blue', 'white', 'green']}
Map.addLayer(ndre, ndreParams, 'NDRE Image')

addNDRE = (lambda image : image.addBands(ndre))

withNDRE = sentinel2.map(addNDRE)

In [253]:
#Make chart -- JK ui.Chart is only available in the code editor
#chart = ui.Chart.image.series({
#  imageCollection: withNDRE.select('NDRE'),
#  region: brma2,
#  reducer: ee.Reducer.first(),
#  scale: 30
#}).setOptions({title: 'NDRE over time'})
#print(chart)

In [254]:
#Put the EC Tower on top layer
Map.addLayer(brma2, {'color': 'FF0000', 'size': 50}, 'Manauas EC Flux Location')